**This notebook presents code associated with Figure 2 and Figure 2-Supplement of the following paper.**
>Kiichi Watanabe, Hui Chiu, David J. Anderson. "Whole brain in situ mapping of neuronal activation in Drosophila during social behaviors and optogenetic stimulation" eLife (2024) (https://doi.org/10.7554/eLife.92380)
  
**The data**  
The data set (.csv) contains pooled data of 'Population,' 'Behavior,' and 'Hr38_ratio' from individual neurons from multiple individual flies.
Population: types of neurons labeled (P1a, Tk, or aSP2 neurons)
Behavior: behavioral context before detection of Hr38 expression (control, aggression, deadM, NoFood, mesh, or courtship)
Hr38_ratio: the relative expression level of Hr38 (with exon probe)
Fiji/ImageJ was used to measure the Hr38 signal intensity from confocal images. The relative intensity was obtained by dividing by the randomly selected background signal from the same z-plane, then pooled into a single CSV file. For details, please refer to the Materials and Methods section of the paper.

In [ ]:
%matplotlib inline
import pandas as pd
import os
import glob
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats as st

In [ ]:
# the path to your CSV file, and load it into dataframe
csv_file_path = 'specify the path to .csv file' # <-define the path to the csv data file 

df = pd.read_csv(csv_file_path)

# Display the first few rows of the DataFrame to verify it loaded correctly
print(df.head())

**Define thresholds for characterizing neurons based on Hr38 expression level.**  
Calculate the mean and standard deviation of the relative intensity of Hr38 expression for each neuronal population.  
*Criteria for characterizing cell types:*  
Negative: Hr38_ratio < mean + 2xσ  
Borderline positive: mean + 2xσ <= Hr38_ratio < mean + 3xσ  
Positive: Hr38_ratio >= mean + 3xσ  
Based on this criterion for each neuronal population, define 'Cell_type' of each data point.

In [ ]:
# calculate thresholds for borderline and positive populations
control_df = df[df['Behavior'] == 'control']
thresholds = control_df.groupby('Population')['Hr38_Ratio'].agg(['mean', 'std']).reset_index()

thresholds['border'] = thresholds['mean'] + 2 * thresholds['std']
thresholds['positive'] = thresholds['mean'] + 3 * thresholds['std']

print(thresholds)

In [ ]:
# Merge the thresholds DataFrame into df based on 'Population'
df_merged = pd.merge(df, thresholds[['Population', 'border', 'positive']], on='Population', how='left')

# Define the choices for 'Cell_type'
conditions = [
    (df_merged['Hr38_Ratio'] < df_merged['border']),  # 'negative' condition
    (df_merged['Hr38_Ratio'] >= df_merged['border']) & (df_merged['Hr38_Ratio'] < df_merged['positive']),  # 'border' condition
    (df_merged['Hr38_Ratio'] >= df_merged['positive'])  # 'positive' condition
]

choices = ['negative', 'border', 'positive']

# Apply the conditions to create the 'Cell_type' column
df_merged['Cell_type'] = np.select(conditions, choices, default='unknown')

# Display the result to verify
print(df_merged[['Population', 'Hr38_Ratio', 'border', 'positive', 'Cell_type']].head())

**Figure 2B-D -Scatter plot-**  
The upper panels of Figures 2B, 2C, and 2D depict each data point's relative intensity of  Hr38 for different experimental conditions with the indicated color codes.  
Color codes:  
Positive: indian red  
Borderline positive: steel blue  
Negative: gray  

In [ ]:
# Create a scatter graph for P1a
df_P1a = df_merged[df_merged['Population'] == 'P1a']

plt.style.use('ggplot')
fig, ax = plt.subplots(figsize=(8, 6))

color_codes = {'positive': 'indianred', 'negative': 'grey', 'border': 'steelblue'}

ax = sns.stripplot(x='Behavior', y='Hr38_Ratio',data=df_P1a, order=['control','aggression','deadM','NoFood','mesh','courtship'],alpha=0.2, palette=color_codes, hue='Cell_type', s=7.5)
ax.grid(True)
ax.set_title('P1a > myr::GFP')
ax.set_xlabel('Behavior')
ax.set_ylabel('Hr38_Ratio')
#plt.savefig('', dpi=600) # define the name
plt.show()

In [ ]:
# Create a scatter graph for Tk
df_Tk = df_merged[df_merged['Population'] == 'Tk']

plt.style.use('ggplot')
fig, ax = plt.subplots(figsize=(8, 6))

color_codes = {'positive': 'indianred', 'negative': 'grey', 'border': 'steelblue'}

ax = sns.stripplot(x='Behavior', y='Hr38_Ratio',data=df_Tk, order=['control','aggression','deadM','NoFood','mesh','courtship'],alpha=0.2, palette=color_codes, hue='Cell_type', s=7.5)
ax.grid(True)
ax.set_title('Tk > myr::GFP')
ax.set_xlabel('Behavior')
ax.set_ylabel('Hr38_Ratio')
#plt.savefig('', dpi=600) # define the name
plt.show()

In [ ]:
# Create a scatter graph for aSP2
df_aSP2 = df_merged[df_merged['Population'] == 'aSP2']

plt.style.use('ggplot')
fig, ax = plt.subplots(figsize=(8, 6))

color_codes = {'positive': 'indianred', 'negative': 'grey', 'border': 'steelblue'}

ax = sns.stripplot(x='Behavior', y='Hr38_Ratio',data=df_aSP2, order=['control','aggression','deadM','NoFood','mesh','courtship'],alpha=0.2, palette=color_codes, hue='Cell_type', s=7.5)
ax.grid(True)
ax.set_title('aSP2 > myr::GFP')
ax.set_xlabel('Behavior')
ax.set_ylabel('Hr38_Ratio')
#plt.savefig('', dpi=600) # define the name
plt.show()

**Create a table containing the number of Hr38 positive population and Hr38 negative population for each experimental condition.**  
In this study, we only included 'positive' as 'Hr38 Positive' (we did not include 'border'). 

In [ ]:
# P1a stats Positive vs Negative
behaviors = ['control', 'aggression', 'deadM', 'NoFood', 'mesh', 'courtship']

# Initialize lists to store positive and negative counts
Hr38_Positive = []
Hr38_Negative = []

for behavior in behaviors:
    # Total number of instances for this behavior
    Num_behavior_total = (df_P1a['Behavior'] == behavior).sum()
    
    # Number of positive Hr38 for this behavior
    Num_behavior_Hr38_Pos = ((df_P1a['Behavior'] == behavior) & (df_P1a['Cell_type'] == 'positive')).sum()
    
    # Calculate number of negative Hr38 for this behavior
    Num_behavior_Hr38_Neg = Num_behavior_total - Num_behavior_Hr38_Pos
    
    # Append the results to the lists
    Hr38_Positive.append(Num_behavior_Hr38_Pos)
    Hr38_Negative.append(Num_behavior_Hr38_Neg)

# Create a DataFrame from the results
P1a_Chi = pd.DataFrame({
    'Hr38_Positive': Hr38_Positive,
    'Hr38_Negative': Hr38_Negative
}, index=['Control', 'Aggression', 'deadM', 'NoFood', 'Mesh', 'Courtship'])

# Display the DataFrame
print(P1a_Chi)

**Figure 2B-D -Bar graph-**  
The graphs depict the percentage of Hr38 positive neurons in a specific population after different behavioral episodes.

In [ ]:
# Plot a bar graph
P1a_Chi['Hr38_Positive_Ratio'] = P1a_Chi['Hr38_Positive'] / (P1a_Chi['Hr38_Positive'] + P1a_Chi['Hr38_Negative'])

plt.style.use('default')
plt.figure(figsize=(8, 6))

sns.barplot(x=P1a_Chi.index, y=P1a_Chi['Hr38_Positive_Ratio'], color='gray', width=0.5)

plt.title('Hr38 Positive Ratio for Each Condition (P1a Population)', fontsize=14)
plt.xlabel('Condition', fontsize=12)
plt.ylabel('Hr38 Positive Ratio', fontsize=12)
plt.ylim(0, 1) 

plt.xticks(rotation=45)

#plt.savefig('', dpi=600) # define the name
plt.show()

**Perform a chi-square test between two behavioral conditions.**  
To compare two behavioral conditions, a chi-square test of independence was performed for all possible combinations of conditions. Following the initial tests, a Bonferroni correction for multiple comparisons was applied for Type I errors. Behavioral conditions that show significant differences from the control group after correction were manually selected and used for further analysis for Figure S2B-D.

In [ ]:
P1a_vs_agg = P1a_Chi.loc['Control':'Aggression',:]
x2, p, dof, e = st.chi2_contingency(P1a_vs_agg)
print('P1a / vs aggression')
print(f'p-value = {p:.5f}')
print(f'chi-square = {x2:.2f}')
print(f'freedom = {dof}')
print('')

P1a_vs_deadM = P1a_Chi.loc[['Control','deadM'],:]
x2, p, dof, e = st.chi2_contingency(P1a_vs_deadM)
print('P1a / vs dead male')
print(f'p-value = {p:.5f}')
print(f'chi-square = {x2:.2f}')
print(f'freedom = {dof}')
print('')

P1a_vs_NoFood = P1a_Chi.loc[['Control','NoFood'],:]
x2, p, dof, e = st.chi2_contingency(P1a_vs_NoFood)
print('P1a / vs No Food')
print(f'p-value = {p:.5f}')
print(f'chi-square = {x2:.2f}')
print(f'freedom = {dof}')
print('')

P1a_vs_mesh = P1a_Chi.loc[['Control','Mesh'],:]
x2, p, dof, e = st.chi2_contingency(P1a_vs_mesh)
print('P1a / vs mesh')
print(f'p-value = {p:.5f}')
print(f'chi-square = {x2:.2f}')
print(f'freedom = {dof}')
print('')

P1a_vs_courtship = P1a_Chi.loc[['Control','Courtship'],:]
x2, p, dof, e = st.chi2_contingency(P1a_vs_courtship)
print('P1a / vs courtship')
print(f'p-value = {p:.5f}')
print(f'chi-square = {x2:.2f}')
print(f'freedom = {dof}')
print('')

P1a_agg_vs_deadM = P1a_Chi.loc[['Aggression','deadM'],:]
x2, p, dof, e = st.chi2_contingency(P1a_agg_vs_deadM)
print('P1a / aggression vs dead male')
print(f'p-value = {p:.5f}')
print(f'chi-square = {x2:.2f}')
print(f'freedom = {dof}')
print('')

P1a_agg_vs_NoFood = P1a_Chi.loc[['Aggression','NoFood'],:]
x2, p, dof, e = st.chi2_contingency(P1a_agg_vs_NoFood)
print('P1a / aggression vs No Food')
print(f'p-value = {p:.5f}')
print(f'chi-square = {x2:.2f}')
print(f'freedom = {dof}')
print('')

P1a_agg_vs_mesh = P1a_Chi.loc[['Aggression','Mesh'],:]
x2, p, dof, e = st.chi2_contingency(P1a_agg_vs_mesh)
print('P1a / aggression vs mesh')
print(f'p-value = {p:.5f}')
print(f'chi-square = {x2:.2f}')
print(f'freedom = {dof}')
print('')

P1a_agg_vs_courtship = P1a_Chi.loc[['Aggression','Courtship'],:]
x2, p, dof, e = st.chi2_contingency(P1a_agg_vs_courtship)
print('P1a / aggression vs courtship')
print(f'p-value = {p:.5f}')
print(f'chi-square = {x2:.2f}')
print(f'freedom = {dof}')
print('')

P1a_deadM_vs_NoFood = P1a_Chi.loc[['deadM','NoFood'],:]
x2, p, dof, e = st.chi2_contingency(P1a_deadM_vs_NoFood)
print('P1a / dead male vs No Food')
print(f'p-value = {p:.5f}')
print(f'chi-square = {x2:.2f}')
print(f'freedom = {dof}')
print('')

P1a_deadM_vs_mesh = P1a_Chi.loc[['deadM','Mesh'],:]
x2, p, dof, e = st.chi2_contingency(P1a_deadM_vs_mesh)
print('P1a / dead male vs mesh')
print(f'p-value = {p:.5f}')
print(f'chi-square = {x2:.2f}')
print(f'freedom = {dof}')
print('')

P1a_deadM_vs_courtship = P1a_Chi.loc[['deadM','Courtship'],:]
x2, p, dof, e = st.chi2_contingency(P1a_deadM_vs_courtship)
print('P1a / dead male vs courtship')
print(f'p-value = {p:.5f}')
print(f'chi-square = {x2:.2f}')
print(f'freedom = {dof}')
print('')

P1a_NoFood_vs_mesh = P1a_Chi.loc[['NoFood','Mesh'],:]
x2, p, dof, e = st.chi2_contingency(P1a_NoFood_vs_mesh)
print('P1a / No Food vs mesh')
print(f'p-value = {p:.5f}')
print(f'chi-square = {x2:.2f}')
print(f'freedom = {dof}')
print('')

P1a_NoFood_vs_courtship = P1a_Chi.loc[['NoFood','Courtship'],:]
x2, p, dof, e = st.chi2_contingency(P1a_NoFood_vs_courtship)
print('P1a / No Food vs courtship')
print(f'p-value = {p:.5f}')
print(f'chi-square = {x2:.2f}')
print(f'freedom = {dof}')
print('')

P1a_mesh_vs_courtship = P1a_Chi.loc[['Mesh','Courtship'],:]
x2, p, dof, e = st.chi2_contingency(P1a_mesh_vs_courtship)
print('P1a / mesh vs courtship')
print(f'p-value = {p:.5f}')
print(f'chi-square = {x2:.2f}')
print(f'freedom = {dof}')
print('')

In [ ]:
# filter only conditions different from control, and Hr38 relative intensity of positive population
# create a box plot overlayed with scatter
behaviors_of_interest = ['aggression', 'deadM', 'NoFood', 'mesh', 'courtship']
df_P1a_filtered = df_P1a[df_P1a['Behavior'].isin(behaviors_of_interest)]

plt.style.use('default')
fig, ax = plt.subplots()

ax = sns.boxplot(x='Behavior', y='Hr38_Ratio',data=df_P1a_filtered[df_P1a_filtered['Cell_type']=='positive'], order=['aggression','deadM','NoFood','mesh','courtship'], width=0.5, showfliers=False, boxprops={'facecolor':'lightgray', 'edgecolor':'gray'})
ax = sns.stripplot(x='Behavior', y='Hr38_Ratio', data=df_P1a_filtered[df_P1a_filtered['Cell_type']=='positive'], order=['aggression','deadM','NoFood','mesh','courtship'], color='steelblue', jitter=True,)
 
ax.set_ylim(0,20.5)
plt.title('Hr38 Ratio of positive population by Behavior (Filtered Population)')
plt.xlabel('Behavior')
plt.ylabel('Hr38 Ratio')

#plt.savefig('', dpi=600) # define the name
plt.show()

In [ ]:
# Tk stats Positive vs Negative
behaviors = ['control', 'aggression', 'deadM', 'NoFood', 'mesh', 'courtship']

# Initialize lists to store positive and negative counts
Hr38_Positive = []
Hr38_Negative = []

for behavior in behaviors:
    # Total number of instances for this behavior
    Num_behavior_total = (df_Tk['Behavior'] == behavior).sum()
    
    # Number of positive Hr38 for this behavior
    Num_behavior_Hr38_Pos = ((df_Tk['Behavior'] == behavior) & (df_Tk['Cell_type'] == 'positive')).sum()
    
    # Calculate number of negative Hr38 for this behavior
    Num_behavior_Hr38_Neg = Num_behavior_total - Num_behavior_Hr38_Pos
    
    # Append the results to the lists
    Hr38_Positive.append(Num_behavior_Hr38_Pos)
    Hr38_Negative.append(Num_behavior_Hr38_Neg)

# Create a DataFrame from the results
Tk_Chi = pd.DataFrame({
    'Hr38_Positive': Hr38_Positive,
    'Hr38_Negative': Hr38_Negative
}, index=['Control', 'Aggression', 'deadM', 'NoFood', 'Mesh', 'Courtship'])

# Display the DataFrame
print(Tk_Chi)

In [ ]:
# Plot a bar graph
Tk_Chi['Hr38_Positive_Ratio'] = Tk_Chi['Hr38_Positive'] / (Tk_Chi['Hr38_Positive'] + Tk_Chi['Hr38_Negative'])

plt.style.use('default')
plt.figure(figsize=(8, 6))

sns.barplot(x=Tk_Chi.index, y=Tk_Chi['Hr38_Positive_Ratio'], color='gray', width=0.5)

plt.title('Hr38 Positive Ratio for Each Condition (Tk Population)', fontsize=14)
plt.xlabel('Condition', fontsize=12)
plt.ylabel('Hr38 Positive Ratio', fontsize=12)
plt.ylim(0, 0.8)

plt.xticks(rotation=45)

#plt.savefig('', dpi=600) # define the name
plt.show()

In [ ]:
Tk_vs_agg = Tk_Chi.loc['Control':'Aggression',:]
x2, p, dof, e = st.chi2_contingency(Tk_vs_agg)
print('Tk / vs aggression')
print(f'p-value = {p:.5f}')
print(f'chi-square = {x2:.2f}')
print(f'freedom = {dof}')
print('')

Tk_vs_deadM = Tk_Chi.loc[['Control','deadM'],:]
x2, p, dof, e = st.chi2_contingency(Tk_vs_deadM)
print('Tk / vs dead male')
print(f'p-value = {p:.5f}')
print(f'chi-square = {x2:.2f}')
print(f'freedom = {dof}')
print('')

Tk_vs_NoFood = Tk_Chi.loc[['Control','NoFood'],:]
x2, p, dof, e = st.chi2_contingency(Tk_vs_NoFood)
print('Tk / vs No Food')
print(f'p-value = {p:.5f}')
print(f'chi-square = {x2:.2f}')
print(f'freedom = {dof}')
print('')

Tk_vs_mesh = Tk_Chi.loc[['Control','Mesh'],:]
x2, p, dof, e = st.chi2_contingency(Tk_vs_mesh)
print('Tk / vs mesh')
print(f'p-value = {p:.5f}')
print(f'chi-square = {x2:.2f}')
print(f'freedom = {dof}')
print('')

Tk_vs_courtship = Tk_Chi.loc[['Control','Courtship'],:]
x2, p, dof, e = st.chi2_contingency(Tk_vs_courtship)
print('Tk / vs courtship')
print(f'p-value = {p:.5f}')
print(f'chi-square = {x2:.2f}')
print(f'freedom = {dof}')
print('')

Tk_agg_vs_deadM = Tk_Chi.loc[['Aggression','deadM'],:]
x2, p, dof, e = st.chi2_contingency(Tk_agg_vs_deadM)
print('Tk / aggression vs dead male')
print(f'p-value = {p:.5f}')
print(f'chi-square = {x2:.2f}')
print(f'freedom = {dof}')
print('')

Tk_agg_vs_NoFood = Tk_Chi.loc[['Aggression','NoFood'],:]
x2, p, dof, e = st.chi2_contingency(Tk_agg_vs_NoFood)
print('Tk / aggression vs No Food')
print(f'p-value = {p:.5f}')
print(f'chi-square = {x2:.2f}')
print(f'freedom = {dof}')
print('')

Tk_agg_vs_mesh = Tk_Chi.loc[['Aggression','Mesh'],:]
x2, p, dof, e = st.chi2_contingency(Tk_agg_vs_mesh)
print('Tk / aggression vs mesh')
print(f'p-value = {p:.5f}')
print(f'chi-square = {x2:.2f}')
print(f'freedom = {dof}')
print('')

Tk_agg_vs_courtship = Tk_Chi.loc[['Aggression','Courtship'],:]
x2, p, dof, e = st.chi2_contingency(Tk_agg_vs_courtship)
print('Tk / aggression vs courtship')
print(f'p-value = {p:.5f}')
print(f'chi-square = {x2:.2f}')
print(f'freedom = {dof}')
print('')

Tk_deadM_vs_NoFood = Tk_Chi.loc[['deadM','NoFood'],:]
x2, p, dof, e = st.chi2_contingency(Tk_deadM_vs_NoFood)
print('Tk / dead male vs No Food')
print(f'p-value = {p:.5f}')
print(f'chi-square = {x2:.2f}')
print(f'freedom = {dof}')
print('')

Tk_deadM_vs_mesh = Tk_Chi.loc[['deadM','Mesh'],:]
x2, p, dof, e = st.chi2_contingency(Tk_deadM_vs_mesh)
print('Tk / dead male vs mesh')
print(f'p-value = {p:.5f}')
print(f'chi-square = {x2:.2f}')
print(f'freedom = {dof}')
print('')

Tk_deadM_vs_courtship = Tk_Chi.loc[['deadM','Courtship'],:]
x2, p, dof, e = st.chi2_contingency(Tk_deadM_vs_courtship)
print('Tk / dead male vs courtship')
print(f'p-value = {p:.5f}')
print(f'chi-square = {x2:.2f}')
print(f'freedom = {dof}')
print('')

Tk_NoFood_vs_mesh = Tk_Chi.loc[['NoFood','Mesh'],:]
x2, p, dof, e = st.chi2_contingency(Tk_NoFood_vs_mesh)
print('Tk / No Food vs mesh')
print(f'p-value = {p:.5f}')
print(f'chi-square = {x2:.2f}')
print(f'freedom = {dof}')
print('')

Tk_NoFood_vs_courtship = Tk_Chi.loc[['NoFood','Courtship'],:]
x2, p, dof, e = st.chi2_contingency(Tk_NoFood_vs_courtship)
print('Tk / No Food vs courtship')
print(f'p-value = {p:.5f}')
print(f'chi-square = {x2:.2f}')
print(f'freedom = {dof}')
print('')

Tk_mesh_vs_courtship = Tk_Chi.loc[['Mesh','Courtship'],:]
x2, p, dof, e = st.chi2_contingency(Tk_mesh_vs_courtship)
print('Tk / mesh vs courtship')
print(f'p-value = {p:.5f}')
print(f'chi-square = {x2:.2f}')
print(f'freedom = {dof}')
print('')

In [ ]:
# filter only conditions different from control, and Hr38 relative intensity of positive population
behaviors_of_interest = ['aggression', 'deadM', 'NoFood']
df_Tk_filtered = df_Tk[df_Tk['Behavior'].isin(behaviors_of_interest)]

plt.style.use('default')
fig, ax = plt.subplots()

ax = sns.boxplot(x='Behavior', y='Hr38_Ratio',data=df_Tk_filtered[df_Tk_filtered['Cell_type']=='positive'], order=['aggression','deadM','NoFood'], width=0.5, showfliers=False, boxprops={'facecolor':'lightgray', 'edgecolor':'gray'})
ax = sns.stripplot(x='Behavior', y='Hr38_Ratio', data=df_Tk_filtered[df_Tk_filtered['Cell_type']=='positive'], order=['aggression','deadM','NoFood'], color='steelblue', jitter=True,)
 
ax.set_ylim(0,40.5)
plt.title('Hr38 Ratio of positive population by Behavior (Filtered Population)')
plt.xlabel('Behavior')
plt.ylabel('Hr38 Ratio')

#plt.savefig('', dpi=600) # define the name
plt.show()

In [ ]:
# aSP2 stats Positive vs Negative
behaviors = ['control', 'aggression', 'deadM', 'NoFood', 'mesh', 'courtship']

# Initialize lists to store positive and negative counts
Hr38_Positive = []
Hr38_Negative = []

for behavior in behaviors:
    # Total number of instances for this behavior
    Num_behavior_total = (df_aSP2['Behavior'] == behavior).sum()
    
    # Number of positive Hr38 for this behavior
    Num_behavior_Hr38_Pos = ((df_aSP2['Behavior'] == behavior) & (df_aSP2['Cell_type'] == 'positive')).sum()
    
    # Calculate number of negative Hr38 for this behavior
    Num_behavior_Hr38_Neg = Num_behavior_total - Num_behavior_Hr38_Pos
    
    # Append the results to the lists
    Hr38_Positive.append(Num_behavior_Hr38_Pos)
    Hr38_Negative.append(Num_behavior_Hr38_Neg)

# Create a DataFrame from the results
aSP2_Chi = pd.DataFrame({
    'Hr38_Positive': Hr38_Positive,
    'Hr38_Negative': Hr38_Negative
}, index=['Control', 'Aggression', 'deadM', 'NoFood', 'Mesh', 'Courtship'])

# Display the DataFrame
print(aSP2_Chi)

In [ ]:
# Plot a bar graph
aSP2_Chi['Hr38_Positive_Ratio'] = aSP2_Chi['Hr38_Positive'] / (aSP2_Chi['Hr38_Positive'] + aSP2_Chi['Hr38_Negative'])

plt.style.use('default')
plt.figure(figsize=(8, 6))

sns.barplot(x=aSP2_Chi.index, y=aSP2_Chi['Hr38_Positive_Ratio'], color='gray', width=0.5)

plt.title('Hr38 Positive Ratio for Each Condition (aSP2 Population)', fontsize=14)
plt.xlabel('Condition', fontsize=12)
plt.ylabel('Hr38 Positive Ratio', fontsize=12)
plt.ylim(0, 1) 

plt.xticks(rotation=45)

#plt.savefig('', dpi=600) # define the name
plt.show()

In [ ]:
aSP2_vs_agg = aSP2_Chi.loc['Control':'Aggression',:]
x2, p, dof, e = st.chi2_contingency(aSP2_vs_agg)
print('aSP2 / vs aggression')
print(f'p-value = {p:.5f}')
print(f'chi-square = {x2:.2f}')
print(f'freedom = {dof}')
print('')

aSP2_vs_deadM = aSP2_Chi.loc[['Control','deadM'],:]
x2, p, dof, e = st.chi2_contingency(aSP2_vs_deadM)
print('aSP2 / vs dead male')
print(f'p-value = {p:.5f}')
print(f'chi-square = {x2:.2f}')
print(f'freedom = {dof}')
print('')

aSP2_vs_NoFood = aSP2_Chi.loc[['Control','NoFood'],:]
x2, p, dof, e = st.chi2_contingency(aSP2_vs_NoFood)
print('aSP2 / vs No Food')
print(f'p-value = {p:.5f}')
print(f'chi-square = {x2:.2f}')
print(f'freedom = {dof}')
print('')

aSP2_vs_mesh = aSP2_Chi.loc[['Control','Mesh'],:]
x2, p, dof, e = st.chi2_contingency(aSP2_vs_mesh)
print('aSP2 / vs mesh')
print(f'p-value = {p:.5f}')
print(f'chi-square = {x2:.2f}')
print(f'freedom = {dof}')
print('')

aSP2_vs_courtship = aSP2_Chi.loc[['Control','Courtship'],:]
x2, p, dof, e = st.chi2_contingency(aSP2_vs_courtship)
print('aSP2 / vs courtship')
print(f'p-value = {p:.5f}')
print(f'chi-square = {x2:.2f}')
print(f'freedom = {dof}')
print('')

aSP2_agg_vs_deadM = aSP2_Chi.loc[['Aggression','deadM'],:]
x2, p, dof, e = st.chi2_contingency(aSP2_agg_vs_deadM)
print('aSP2 / aggression vs dead male')
print(f'p-value = {p:.5f}')
print(f'chi-square = {x2:.2f}')
print(f'freedom = {dof}')
print('')

aSP2_agg_vs_NoFood = aSP2_Chi.loc[['Aggression','NoFood'],:]
x2, p, dof, e = st.chi2_contingency(aSP2_agg_vs_NoFood)
print('aSP2 / aggression vs No Food')
print(f'p-value = {p:.5f}')
print(f'chi-square = {x2:.2f}')
print(f'freedom = {dof}')
print('')

aSP2_agg_vs_mesh = aSP2_Chi.loc[['Aggression','Mesh'],:]
x2, p, dof, e = st.chi2_contingency(aSP2_agg_vs_mesh)
print('aSP2 / aggression vs mesh')
print(f'p-value = {p:.5f}')
print(f'chi-square = {x2:.2f}')
print(f'freedom = {dof}')
print('')

aSP2_agg_vs_courtship = aSP2_Chi.loc[['Aggression','Courtship'],:]
x2, p, dof, e = st.chi2_contingency(aSP2_agg_vs_courtship)
print('aSP2 / aggression vs courtship')
print(f'p-value = {p:.5f}')
print(f'chi-square = {x2:.2f}')
print(f'freedom = {dof}')
print('')

aSP2_deadM_vs_NoFood = aSP2_Chi.loc[['deadM','NoFood'],:]
x2, p, dof, e = st.chi2_contingency(aSP2_deadM_vs_NoFood)
print('aSP2 / dead male vs No Food')
print(f'p-value = {p:.5f}')
print(f'chi-square = {x2:.2f}')
print(f'freedom = {dof}')
print('')

aSP2_deadM_vs_mesh = aSP2_Chi.loc[['deadM','Mesh'],:]
x2, p, dof, e = st.chi2_contingency(aSP2_deadM_vs_mesh)
print('aSP2 / dead male vs mesh')
print(f'p-value = {p:.5f}')
print(f'chi-square = {x2:.2f}')
print(f'freedom = {dof}')
print('')

aSP2_deadM_vs_courtship = aSP2_Chi.loc[['deadM','Courtship'],:]
x2, p, dof, e = st.chi2_contingency(aSP2_deadM_vs_courtship)
print('aSP2 / dead male vs courtship')
print(f'p-value = {p:.5f}')
print(f'chi-square = {x2:.2f}')
print(f'freedom = {dof}')
print('')

aSP2_NoFood_vs_mesh = aSP2_Chi.loc[['NoFood','Mesh'],:]
x2, p, dof, e = st.chi2_contingency(aSP2_NoFood_vs_mesh)
print('aSP2 / No Food vs mesh')
print(f'p-value = {p:.5f}')
print(f'chi-square = {x2:.2f}')
print(f'freedom = {dof}')
print('')

aSP2_NoFood_vs_courtship = aSP2_Chi.loc[['NoFood','Courtship'],:]
x2, p, dof, e = st.chi2_contingency(aSP2_NoFood_vs_courtship)
print('aSP2 / No Food vs courtship')
print(f'p-value = {p:.5f}')
print(f'chi-square = {x2:.2f}')
print(f'freedom = {dof}')
print('')

aSP2_mesh_vs_courtship = aSP2_Chi.loc[['Mesh','Courtship'],:]
x2, p, dof, e = st.chi2_contingency(aSP2_mesh_vs_courtship)
print('aSP2 / mesh vs courtship')
print(f'p-value = {p:.5f}')
print(f'chi-square = {x2:.2f}')
print(f'freedom = {dof}')
print('')

In [ ]:
# filter only conditions different from control, and Hr38 relative intensity of positive population
behaviors_of_interest = ['aggression', 'deadM', 'NoFood', 'courtship']
df_aSP2_filtered = df_aSP2[df_aSP2['Behavior'].isin(behaviors_of_interest)]

plt.style.use('default')
fig, ax = plt.subplots()

ax = sns.boxplot(x='Behavior', y='Hr38_Ratio',data=df_aSP2_filtered[df_aSP2_filtered['Cell_type']=='positive'], order=['aggression','deadM','NoFood','courtship'], width=0.5, showfliers=False, boxprops={'facecolor':'lightgray', 'edgecolor':'gray'})
ax = sns.stripplot(x='Behavior', y='Hr38_Ratio', data=df_aSP2_filtered[df_aSP2_filtered['Cell_type']=='positive'], order=['aggression','deadM','NoFood','courtship'], color='steelblue', jitter=True,)
 
ax.set_ylim(0,65.5)
plt.title('Hr38 Ratio of positive population by Behavior (Filtered Population)')
plt.xlabel('Behavior')
plt.ylabel('Hr38 Ratio')
plt.show()